This is a modified version of the official notebook, adapted to be fully compatible and runnable on Google Colab. Adjustments have been made to ensure smooth execution in the Colab environment.

### Install GitHub repo Packages

In [ ]:
!git clone https://github.com/dxd1019/deep-regression-unlearning

%cd deep-regression-unlearning

Cloning into 'deep-regression-unlearning'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 78 (delta 35), reused 43 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 87.27 KiB | 3.23 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/deep-regression-unlearning


In [ ]:
!pip install requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.3/86.3 kB 3.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e77325a3ae633089d2bb6e57502fc317bcd4e7fc35d5c1ac3f304feb21d8954a
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [ ]:
import urllib.request

In [ ]:
!ls
%cd deep-regression-unlearning

AgeDB_AllCNN_0to30_forgetting.ipynb  LICENSE	       script_download_data.py
AgeDB_AllCNN_0to30_forgetting.py     metrics.py        STSB_LSTM_Unlearn_0_to_2.ipynb
data_formatters			     models.py	       STSB_LSTM_Unlearn_0_to_2.py
datasets.py			     README.md	       unlearn.py
expt_settings			     requirements.txt  utils.py
[Errno 2] No such file or directory: 'deep-regression-unlearning'
/content/deep-regression-unlearning


In [ ]:
# %run script_download_data.py electricity ./data no

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7b416270cfe0>
Traceback (most recent call last):
  File "/usr/lib/python3.11/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import torch
import os
import wget
import zipfile
import re

import numpy as np
import pandas as pd
from torch.nn import functional as F
from utils import clean_text
from models import LSTMnetwork

## Define some helper functions

In [ ]:
def training_step(model, batch, device):
    sent1, sent2, labels = batch
    sent1, sent2, labels = sent1.to(device), sent2.to(device), labels.to(device)
    out, *_  = model(sent1, sent2)                  # Generate predictions
    loss= F.mse_loss(out, labels) # Calculate loss
    return loss

def validation_step(model, batch, device):
    sent1, sent2, labels = batch
    sent1, sent2, labels = sent1.to(device), sent2.to(device), labels.to(device)
    out, *_  = model(sent1, sent2)                    # Generate predictions
    loss= F.mse_loss(out, labels)   # Calculate loss
    return {'Loss': loss.detach()}

def validation_epoch_end(model, outputs):
    batch_losses = [x['Loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    return {'Loss': epoch_loss.item()}

def epoch_end(model, epoch, result):
    print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(
        epoch, result['lrs'][-1], result['train_loss'], result['Loss']))



@torch.no_grad()
def evaluate(model, val_df, device, batch_size = 256):
    model.eval()
    outputs = []

    num_steps = len(val_df)//batch_size

    for i in range(num_steps):
        sent1 = torch.tensor(np.stack(val_df.iloc[i*batch_size:(i+1)*batch_size]['sentence1'])).float()
        sent2 = torch.tensor(np.stack(val_df.iloc[i*batch_size:(i+1)*batch_size]['sentence2'])).float()
        labels = torch.tensor(val_df.iloc[i*batch_size:(i+1)*batch_size]['score'].values)
        batch = (sent1, sent2, labels)

        outputs.append(validation_step(model, batch, device))

    if len(val_df)%batch_size != 0:
        sent1 = torch.tensor(np.stack(val_df.iloc[num_steps*batch_size:]['sentence1'])).float()
        sent2 = torch.tensor(np.stack(val_df.iloc[num_steps*batch_size:]['sentence2'])).float()
        labels = torch.tensor(val_df.iloc[num_steps*batch_size:]['score'].values)
        batch = (sent1, sent2, labels)

        outputs.append(validation_step(model, batch, device))

    return validation_epoch_end(model, outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs,  model, train_df, val_df, device, save_path, batch_size = 256):
    best_loss = np.inf
    torch.cuda.empty_cache()
    history = []

    optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
    num_steps = len(train_df)//batch_size

    #for
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for i in range(num_steps):
            sent1 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[i*batch_size:(i+1)*batch_size]['score'].values).float()
            batch = (sent1, sent2, labels)
            loss = training_step(model, batch, device)
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))
        if len(train_df)%batch_size != 0:
            sent1 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[num_steps*batch_size:]['score'].values).float()
            batch = (sent1, sent2, labels)
            loss = training_step(model, batch, device)
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))


        # Validation phase
        result = evaluate(model, val_df, device, batch_size)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        epoch_end(model, epoch, result)
        history.append(result)
        sched.step(result['Loss'])
        if best_loss > result['Loss']:
            best_loss = result['Loss']
            torch.save(model.state_dict(), save_path)

    return history

In [ ]:
def fit_one_finetune_cycle(epochs,  model, train_df, val_df, lr, device, save_path, batch_size = 256):
    best_loss = np.inf
    torch.cuda.empty_cache()
    history = []

    optimizer = torch.optim.Adam(model.parameters(), lr = lr)

    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
    num_steps = len(train_df)//batch_size

    #for
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for i in range(num_steps):
            sent1 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[i*batch_size:(i+1)*batch_size]['score'].values).float()
            batch = (sent1, sent2, labels)
            loss = training_step(model, batch, device)
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))
        if len(train_df)%batch_size != 0:
            sent1 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[num_steps*batch_size:]['score'].values).float()
            batch = (sent1, sent2, labels)
            loss = training_step(model, batch, device)
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))


        # Validation phase
        result = evaluate(model, val_df, device, batch_size)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        epoch_end(model, epoch, result)
        history.append(result)
        sched.step(result['Loss'])
        if best_loss > result['Loss']:
            best_loss = result['Loss']
            torch.save(model.state_dict(), save_path)

    return history

In [ ]:
def attention(x):
        """
        Taken from https://github.com/szagoruyko/attention-transfer
        :param x = activations
        """
        return F.normalize(x.pow(2).mean(1).view(x.size(0), -1))


def attention_diff(x, y):
    """
    Taken from https://github.com/szagoruyko/attention-transfer
    :param x = activations
    :param y = activations
    """
    return (attention(x) - attention(y)).pow(2).mean()



def forget_loss(model_output, model_activations, proxy_output, proxy_activations, mask):

    loss = F.mse_loss(model_output[mask], proxy_output[mask])
    if AT_beta > 0:
        at_loss = 0
        for i in range(len(proxy_activations)):
            at_loss = at_loss + AT_beta * attention_diff(model_activations[i][mask], proxy_activations[i][mask])
    else:
        at_loss = 0

    total_loss = loss + at_loss

    return total_loss



def fit_one_forget_cycle(epochs,  model, proxy_model, train_df, val_df, lr, device, save_path, batch_size = 256):
    best_loss = np.inf
    torch.cuda.empty_cache()
    history = []

    optimizer = torch.optim.Adam(model.parameters(), lr = lr)

    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
    num_steps = len(train_df)//batch_size
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        #for batch in train_loader:
        for i in range(num_steps):
            sent1 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[i*batch_size:(i+1)*batch_size]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[i*batch_size:(i+1)*batch_size]['score'].values).float()
            ulabels = torch.tensor(train_df.iloc[i*batch_size:(i+1)*batch_size]['forget'].values)

            sent1, sent2, labels, ulabels = sent1.to(device), sent2.to(device), labels.to(device), ulabels.to(device)

            model_out, *model_activations = model(sent1, sent2)
            with torch.no_grad():
                proxy_out, *proxy_activations = proxy_model(sent1, sent2)


            label_loss = 0
            if ulabels.sum() < len(ulabels):
                mask = (ulabels == 0)
                r_model_out = model_out[mask]
                r_labels = labels[mask]
                label_loss = F.mse_loss(r_model_out, r_labels)

            proxy_loss = 0
            if ulabels.sum() > 0:
                mask = (ulabels == 1)
                proxy_loss = forget_loss(model_out, model_activations, proxy_out, proxy_activations, mask)

            coeff = ulabels.sum()/len(ulabels)
            loss = coeff*proxy_loss + (1-coeff)*label_loss

            ######
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))

        if len(train_df)%batch_size != 0:
            sent1 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence1'])).float()
            sent2 = torch.tensor(np.stack(train_df.iloc[num_steps*batch_size:]['sentence2'])).float()
            labels = torch.tensor(train_df.iloc[num_steps*batch_size:]['score'].values).float()
            ulabels = torch.tensor(train_df.iloc[num_steps*batch_size:]['forget'].values)

            sent1, sent2, labels, ulabels = sent1.to(device), sent2.to(device), labels.to(device), ulabels.to(device)

            model_out, *model_activations = model(sent1, sent2)
            with torch.no_grad():
                proxy_out, *proxy_activations = proxy_model(sent1, sent2)


            label_loss = 0
            if ulabels.sum() < len(ulabels):
                mask = (ulabels == 0)
                r_model_out = model_out[mask]
                r_labels = labels[mask]
                label_loss = F.mse_loss(r_model_out, r_labels)

            proxy_loss = 0
            if ulabels.sum() > 0:
                mask = (ulabels == 1)
                proxy_loss = forget_loss(model_out, model_activations, proxy_out, proxy_activations, mask)

            coeff = ulabels.sum()/len(ulabels)
            loss = coeff*proxy_loss + (1-coeff)*label_loss

            ######
            train_losses.append(loss)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            lrs.append(get_lr(optimizer))


        # Validation phase
        result = evaluate(model, val_df, device)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        epoch_end(model, epoch, result)
        history.append(result)
        #sched.step(result['Loss'])
        #if best_loss > result['Loss']:
        #    best_loss = result['Loss']
        #    torch.save(model.state_dict(), save_path)
        torch.save(model.state_dict(), save_path)

    return history

In [ ]:
text_embedding_dimension = 300

def text_embed(words):

    unknown_indices = []
    mean = np.zeros(text_embedding_dimension)

    for i in range(len(words)):
        if words[i] in embeddings_index_300 and embeddings_index_300[ words[i] ].shape == (300, ):
            words[i] = embeddings_index_300[ words[i] ]
            mean += words[i]
        else:
            unknown_indices.append(i)

    mean /= max(len(words)-len(unknown_indices), 1)

    # unknown words in the text are represented using the mean of the known words
    for i in unknown_indices:
        words[i] = mean
    return words

def pad(x, max_len = 10):
    if len(x) >= max_len:
        return x[:10]
    zeros = [np.zeros(text_embedding_dimension)]*(max_len - len(x))
    return zeros + x



## Get GLOVE Word embeddings

In [ ]:
print("Downloading and extracting GloVe word embeddings...")
data_file = "./glove.840B.300d.zip"
wget.download("http://nlp.stanford.edu/data/glove.840B.300d.zip", out=data_file)
with zipfile.ZipFile(data_file) as zip_ref:
    zip_ref.extractall('./glove')
os.remove(data_file)
print("\nCompleted!")


Completed!


In [ ]:
path_to_glove_file = "./glove/glove.840B.300d.txt"

embeddings_index_300 = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index_300[word] = coefs

<ipython-input-13-3ba7d9c1b092>:7: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


In [ ]:
print("Found %s word vectors." % len(embeddings_index_300))

Found 2195884 word vectors.


## Load Data

In [ ]:
columns = ["genre", "filename", "year", "old_index", "score", "sentence1", "sentence2"]

train_df = pd.read_csv("./stsb_data/sts-train.tsv", sep='\t', header=None, names=columns, on_bad_lines='skip')
val_df   = pd.read_csv("./stsb_data/sts-dev.tsv", sep='\t', header=None, names=columns, on_bad_lines='skip')
test_df  = pd.read_csv("./stsb_data/sts-test.tsv", sep='\t', header=None, names=columns, quoting=3, on_bad_lines='skip')

In [ ]:
print(test_df.columns)

Index(['genre', 'filename', 'year', 'old_index', 'score', 'sentence1',
       'sentence2'],
      dtype='object')


In [ ]:
train_df.dropna(subset=['score'], inplace=True)
val_df.dropna(subset=['score'], inplace=True)
test_df.dropna(subset=['score'], inplace=True)

# Drop rows with missing sentences
train_df.dropna(subset=['sentence1', 'sentence2'], inplace=True)
val_df.dropna(subset=['sentence1', 'sentence2'], inplace=True)
test_df.dropna(subset=['sentence1', 'sentence2'], inplace=True)

In [ ]:
train_df['sentence1'] = train_df['sentence1'].apply(lambda x: clean_text(x))
train_df['sentence2'] = train_df['sentence2'].apply(lambda x: clean_text(x))

val_df['sentence1'] = val_df['sentence1'].apply(lambda x: clean_text(x))
val_df['sentence2'] = val_df['sentence2'].apply(lambda x: clean_text(x))

test_df['sentence1'] = test_df['sentence1'].apply(lambda x: clean_text(x))
test_df['sentence2'] = test_df['sentence2'].apply(lambda x: clean_text(x))

In [ ]:
train_df['sentence1'] = train_df['sentence1'].apply(lambda words: text_embed(words))
train_df['sentence2'] = train_df['sentence2'].apply(lambda words: text_embed(words))

val_df['sentence1'] = val_df['sentence1'].apply(lambda words: text_embed(words))
val_df['sentence2'] = val_df['sentence2'].apply(lambda words: text_embed(words))

test_df['sentence1'] = test_df['sentence1'].apply(lambda words: text_embed(words))
test_df['sentence2'] = test_df['sentence2'].apply(lambda words: text_embed(words))

In [ ]:
train_df['sentence1'] = train_df['sentence1'].apply(lambda words: pad(words))
train_df['sentence2'] = train_df['sentence2'].apply(lambda words: pad(words))

val_df['sentence1'] = val_df['sentence1'].apply(lambda words: pad(words))
val_df['sentence2'] = val_df['sentence2'].apply(lambda words: pad(words))

test_df['sentence1'] = test_df['sentence1'].apply(lambda words: pad(words))
test_df['sentence2'] = test_df['sentence2'].apply(lambda words: pad(words))

In [ ]:
train_df = train_df.sample(frac = 1, random_state = 0)

## Train the model

In [ ]:
# Dynamically select device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Ensure the directory exists
os.makedirs("saved_models", exist_ok=True)

model = LSTMnetwork(text_embedding_dimension = text_embedding_dimension).to(device)
epochs = 100
save_path = "saved_models/LSTM_STSB_100epochs.pt"
history = fit_one_cycle(epochs, model, train_df, val_df, device = device, save_path = save_path)
model.load_state_dict(torch.load(save_path))

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [0], last_lr: 0.01000, train_loss: 3.4218, val_loss: 2.2629
Epoch [1], last_lr: 0.01000, train_loss: 2.1209, val_loss: 2.2767
Epoch [2], last_lr: 0.01000, train_loss: 2.0348, val_loss: 2.2872
Epoch [3], last_lr: 0.01000, train_loss: 1.9412, val_loss: 2.3165
Epoch [4], last_lr: 0.01000, train_loss: 1.8931, val_loss: 2.3201
Epoch [5], last_lr: 0.01000, train_loss: 1.8103, val_loss: 2.3735
Epoch [6], last_lr: 0.01000, train_loss: 1.7160, val_loss: 2.4143
Epoch [7], last_lr: 0.01000, train_loss: 1.6323, val_loss: 2.4227
Epoch [8], last_lr: 0.01000, train_loss: 1.4831, val_loss: 2.4118
Epoch [9], last_lr: 0.01000, train_loss: 1.3505, val_loss: 2.4400
Epoch [10], last_lr: 0.01000, train_loss: 1.2245, val_loss: 2.4757
Epoch [11], last_lr: 0.01000, train_loss: 1.1064, val_loss: 2.4871
Epoch [12], last_lr: 0.00100, train_loss: 0.9621, val_loss: 2.7062
Epoch [13], last_lr: 0.00100, train_loss: 0.8445, val_loss: 2.8071
Epoch [14], last_lr: 0.00100, train_loss: 0.8097, val_loss: 2.8141
Epoch

<All keys matched successfully>

## Creating the forget and retain sets


In [ ]:
train_df_retain = train_df[train_df['score'] >= 2]
val_df_retain = val_df[val_df['score'] >= 2]
test_df_retain = test_df[test_df['score'] >= 2]

In [ ]:
print(f"Test dataset size: {len(test_df)}")
print(f"Test dataset size: {len(test_df_retain)}")
print(test_df.head)

Test dataset size: 1095
Test dataset size: 752
<bound method NDFrame.head of               genre   filename      year  old_index  score  \
0     main-captions     MSRvid  2012test         24    2.5   
1     main-captions     MSRvid  2012test         33    3.6   
2     main-captions     MSRvid  2012test         45    5.0   
3     main-captions     MSRvid  2012test         63    4.2   
4     main-captions     MSRvid  2012test         66    1.5   
...             ...        ...       ...        ...    ...   
1090      main-news  headlines      2015       1438    0.4   
1091      main-news  headlines      2015       1454    1.4   
1092      main-news  headlines      2015       1456    4.8   
1093      main-news  headlines      2015       1463    4.4   
1094      main-news  headlines      2015       1482    5.0   

                                              sentence1  \
0     [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
1     [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...   
2

In [ ]:
train_df_forget = train_df[train_df['score'] < 2]
val_df_forget = val_df[val_df['score'] < 2]
test_df_forget = test_df[test_df['score'] < 2]

## Retraining the model from scratch on Retain Data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gold_model = LSTMnetwork(text_embedding_dimension=text_embedding_dimension).to(device)

epochs = 100
save_path = "saved_models/LSTM_STSB_100epochs_0to2_retrained.pt"
history = fit_one_cycle(epochs, gold_model, train_df_retain, val_df_retain, device = device, save_path = save_path)
gold_model.load_state_dict(torch.load(save_path))

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [0], last_lr: 0.01000, train_loss: 5.1796, val_loss: 1.0471
Epoch [1], last_lr: 0.01000, train_loss: 1.2543, val_loss: 0.7756
Epoch [2], last_lr: 0.01000, train_loss: 0.8972, val_loss: 0.7321
Epoch [3], last_lr: 0.01000, train_loss: 0.7816, val_loss: 0.7504
Epoch [4], last_lr: 0.01000, train_loss: 0.7462, val_loss: 0.7945
Epoch [5], last_lr: 0.01000, train_loss: 0.7246, val_loss: 0.7950
Epoch [6], last_lr: 0.01000, train_loss: 0.7015, val_loss: 0.7780
Epoch [7], last_lr: 0.01000, train_loss: 0.6921, val_loss: 0.7643
Epoch [8], last_lr: 0.01000, train_loss: 0.6813, val_loss: 0.7745
Epoch [9], last_lr: 0.01000, train_loss: 0.6735, val_loss: 0.7706
Epoch [10], last_lr: 0.01000, train_loss: 0.6671, val_loss: 0.7734
Epoch [11], last_lr: 0.01000, train_loss: 0.6459, val_loss: 0.7634
Epoch [12], last_lr: 0.01000, train_loss: 0.6387, val_loss: 0.7800
Epoch [13], last_lr: 0.01000, train_loss: 0.6254, val_loss: 0.7706
Epoch [14], last_lr: 0.00100, train_loss: 0.6171, val_loss: 0.7553
Epoch

<All keys matched successfully>

### Evaluate the retrained model on various cohorts

In [ ]:
print(f"Test dataset size: {len(test_df_retain)}")

Test dataset size: 752


In [ ]:
# evaluate(model, test_df_retain, 'cuda')
evaluate(model, test_df_retain, 'cpu')

{'Loss': 2.225020241226177}

In [ ]:
# evaluate(model, test_df_forget, 'cuda')
evaluate(model, test_df_retain, 'cpu')

{'Loss': 2.225020241226177}

In [ ]:
# evaluate(gold_model, test_df_retain, 'cuda')
evaluate(model, test_df_retain, 'cpu')

{'Loss': 2.225020241226177}

In [ ]:
# evaluate(gold_model, test_df_forget, 'cuda')
evaluate(model, test_df_retain, 'cpu')

{'Loss': 2.225020241226177}

## Finetuning

In [ ]:
%%time
student_model = LSTMnetwork(text_embedding_dimension = text_embedding_dimension).to(device)
student_model.load_state_dict(torch.load("saved_models/LSTM_STSB_100epochs.pt"))
epochs = 5
save_path = "saved_models/LSTM_STSB_5epochs_4to5_Finetune_Forget.pt"
history = fit_one_finetune_cycle(epochs, student_model, train_df_retain, val_df_retain, 0.001, device = device, save_path = save_path)
student_model.load_state_dict(torch.load(save_path))

Epoch [0], last_lr: 0.00100, train_loss: 1.1413, val_loss: 0.9920
Epoch [1], last_lr: 0.00100, train_loss: 0.7991, val_loss: 0.7721
Epoch [2], last_lr: 0.00100, train_loss: 0.7599, val_loss: 0.7651
Epoch [3], last_lr: 0.00100, train_loss: 0.7407, val_loss: 0.7562
Epoch [4], last_lr: 0.00100, train_loss: 0.7423, val_loss: 0.7424
CPU times: user 6.15 s, sys: 550 ms, total: 6.7 s
Wall time: 7.22 s


<All keys matched successfully>

In [ ]:
# evaluate(student_model, test_df_retain, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7120824645667595}

In [ ]:
# evaluate(student_model, test_df_forget, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7120824645667595}

## Amnesiac Finetuning

In [ ]:
mean = train_df['score'].mean()
sd = train_df['score'].std()

random_preds = np.random.normal(loc=mean, scale=sd, size=(len(train_df[train_df['score'] < 2]),))

amnesiac_finetune_df = train_df.copy()
amnesiac_finetune_df.loc[amnesiac_finetune_df['score'] < 2, 'score'] = random_preds

In [ ]:
%%time
student_model = LSTMnetwork(text_embedding_dimension = text_embedding_dimension).to(device)
student_model.load_state_dict(torch.load("saved_models/LSTM_STSB_100epochs.pt"))
epochs = 5
save_path = "saved_models/LSTM_STSB_2epochs_Amnesiac_Finetune_Forget.pt"
history = fit_one_finetune_cycle(epochs, student_model, amnesiac_finetune_df, val_df_retain, 0.001, device = device, save_path = save_path)
student_model.load_state_dict(torch.load(save_path))

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [0], last_lr: 0.00100, train_loss: 1.5012, val_loss: 0.8694
Epoch [1], last_lr: 0.00100, train_loss: 1.3398, val_loss: 0.7907
Epoch [2], last_lr: 0.00100, train_loss: 1.3200, val_loss: 0.7780
Epoch [3], last_lr: 0.00100, train_loss: 1.2971, val_loss: 0.7691
Epoch [4], last_lr: 0.00100, train_loss: 1.2966, val_loss: 0.7703
CPU times: user 10 s, sys: 843 ms, total: 10.9 s
Wall time: 11.3 s


<All keys matched successfully>

In [ ]:
# evaluate(student_model, test_df_retain, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7337983449954232}

In [ ]:
# evaluate(student_model, test_df_forget, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7337983449954232}

## Blindspot Unlearning

In [ ]:
u_train_df = train_df.copy()

In [ ]:
u_train_df['forget'] = 0
u_train_df.loc[u_train_df['score'] < 2, 'forget'] = 1

### Training the Blindspot model

In [ ]:
%%time
# device = 'cuda'
device = 'cpu'
proxy_model = LSTMnetwork(text_embedding_dimension = text_embedding_dimension).to(device)
epochs = 10
save_path = "saved_models/LSTM_STSB_blindspot.pt"
history = fit_one_cycle(epochs, proxy_model, train_df_retain, val_df_retain, device = device, save_path = save_path)
proxy_model.load_state_dict(torch.load(save_path))

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [0], last_lr: 0.01000, train_loss: 3.7622, val_loss: 1.3992
Epoch [1], last_lr: 0.01000, train_loss: 0.8490, val_loss: 0.7343
Epoch [2], last_lr: 0.01000, train_loss: 0.7439, val_loss: 0.7814
Epoch [3], last_lr: 0.01000, train_loss: 0.7170, val_loss: 0.7425
Epoch [4], last_lr: 0.01000, train_loss: 0.7036, val_loss: 0.7633
Epoch [5], last_lr: 0.01000, train_loss: 0.6868, val_loss: 0.7535
Epoch [6], last_lr: 0.01000, train_loss: 0.6900, val_loss: 0.7506
Epoch [7], last_lr: 0.01000, train_loss: 0.6806, val_loss: 0.7612
Epoch [8], last_lr: 0.01000, train_loss: 0.6602, val_loss: 0.7554
Epoch [9], last_lr: 0.01000, train_loss: 0.6518, val_loss: 0.7533
CPU times: user 13.6 s, sys: 1.59 s, total: 15.2 s
Wall time: 15.3 s


<All keys matched successfully>

In [ ]:
# evaluate(proxy_model, test_df_retain, 'cuda')
evaluate(proxy_model, test_df_retain, 'cpu')

{'Loss': 0.7225442171665678}

In [ ]:
# evaluate(proxy_model, test_df_forget, 'cuda')
evaluate(proxy_model, test_df_retain, 'cpu')

{'Loss': 0.7225442171665678}

In [ ]:
%%time
AT_beta = 50
student_model = LSTMnetwork(text_embedding_dimension = text_embedding_dimension).to(device)
student_model.load_state_dict(torch.load("saved_models/LSTM_STSB_100epochs.pt"))
epochs = 5
save_path = "saved_models/LSTM_STSB_unlearn.pt"
history = fit_one_forget_cycle(epochs, student_model, proxy_model,  u_train_df, val_df, lr = 0.001, device = device, save_path = save_path)
student_model.load_state_dict(torch.load(save_path))

Epoch [0], last_lr: 0.00100, train_loss: 1.2391, val_loss: 3.2372
Epoch [1], last_lr: 0.00100, train_loss: 0.8674, val_loss: 3.6021
Epoch [2], last_lr: 0.00100, train_loss: 0.7069, val_loss: 3.6510
Epoch [3], last_lr: 0.00100, train_loss: 0.6053, val_loss: 3.6217
Epoch [4], last_lr: 0.00100, train_loss: 0.5642, val_loss: 3.6158
CPU times: user 13.6 s, sys: 1.15 s, total: 14.8 s
Wall time: 14.9 s


<All keys matched successfully>

In [ ]:
# evaluate(student_model, test_df_retain, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7163172047360549}

In [ ]:
# evaluate(student_model, test_df_forget, 'cuda')
evaluate(student_model, test_df_retain, 'cpu')

{'Loss': 0.7163172047360549}